In [1]:
import docx
import re
import os
from pathlib import Path
import win32com.client
import sys
from tqdm import tqdm
import pandas as pd

In [10]:
def storeInfo(f, tags, df):
    for tag in list(tags.keys()):
        temp_tag = re.findall("«([^»]*)", tag)
        f.write(temp_tag[0] + "\n")
        # df[tag] = True
        df.loc[len(df.index)-1, temp_tag[0]] = True
    return df

In [3]:
def getTags(paths, rm=False):
    word = win32com.client.Dispatch("Word.Application")
    wdFormatDocumentDefault = 16

    df = pd.DataFrame()

    if paths['bulk']:
        # df = pd.DataFrame()
        for filepath in tqdm(sorted(Path(paths['input']).glob("*.doc"))):
            # if str(filepath).endswith(".doc"):
            doc_file = filepath.parent / filepath.stem
            # print(doc_file, ".docx")
            document = word.Documents.Open(str(filepath))
            document.SaveAs(str(doc_file) + ".docx", FileFormat=wdFormatDocumentDefault)
            document.Close(0)
            if rm:
                os.remove(str(doc_file) + ".doc")
        
        f = open('Info.txt', "w+")
        for filepath in tqdm(sorted(Path(paths['input']).glob("*.docx"))):
            df = df.append({'Name': str(filepath.stem)}, ignore_index=True)
            # print(filepath)
            f.write("\n---- " + str(filepath.stem) + " ----\n")
            document = docx.Document(str(filepath))

            tags = {}
            for line in document.paragraphs:
                # print(line.text)
                # word_lst = []
                for word in line.text.split():
                    # word_lst.append(word)
                    # word_lst.extend(re.findall("«.*»", word))

                    temp = re.findall("«.*»", word)
                    # print(word, temp)
                    try:
                        tags[temp[0]] = tags.get(temp[0], 0) + 1
                    except:
                        # print('---ERROR---')
                        pass
                
                # print(word_lst)
            # print(tags)
            df = storeInfo(f, tags, df)
            # if rm:
            #     os.remove(str(filepath))
            # break
        f.close()
        # return df

    else:
        # df = pd.DataFrame()
        filepath = Path(paths['input'])
        # print(str(filepath))
        if str(filepath).endswith(".doc"):
            # print("Here")
            document = word.Documents.Open(str(filepath))
            document.SaveAs(str(filepath.parent / filepath.stem) + ".docx", FileFormat=wdFormatDocumentDefault)
            document.Close(0)
            if rm:
                os.remove(str(filepath.parent / filepath.stem) + ".doc")
        
        f = open('Info.txt', "w+")
        df = df.append({'Name': str(filepath.stem)}, ignore_index=True)
        f.write("\n---- " + str(filepath.stem) + " ----\n")
        document = docx.Document(str(filepath.parent / filepath.stem) + ".docx")

        tags = {}
        for line in document.paragraphs:
            for word  in line.text.split():
                temp = re.findall("«.*»", word)
                try:
                    tags[temp[0]] = tags.get(temp[0], 0) + 1
                except:
                    pass
        
        df = storeInfo(f, tags, df)
        # if rm:
        #         os.remove(str(filepath.parent / filepath.stem) + ".docx")
        f.close()
        # return df
    return df

In [4]:
def resolvePath(in_path):
    in_path = Path(in_path).resolve()

    paths = {}
    if in_path.is_file():
        print('Getting info from file ...')
        paths['bulk'] = False
        paths['input'] = str(in_path)

    elif in_path.is_dir():
        print('Inside the directory ...')
        paths['bulk'] = True
        paths['input'] = str(in_path)

    else:
        print("Please check your path and try again. Remember to remove the '\\' at the end if it is a folder")
        sys.exit(0)

    return paths

In [11]:
inpath = 'D:\CCS\Template Attachments\myfolder'
paths = resolvePath(inpath)
df = getTags(paths)

Inside the directory ...


0it [00:00, ?it/s]
  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\003XSD744\AppData\Local\Temp\ipykernel_68264\3601875651.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Name': str(filepath.stem)}, ignore_index=True)
C:\Users\003XSD744\AppData\Local\Temp\ipykernel_68264\3601875651.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Name': str(filepath.stem)}, ignore_index=True)
C:\Users\003XSD744\AppData\Local\Temp\ipykernel_68264\3601875651.py:21: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append({'Name': str(filepath.stem)}, ignore_index=True)
C:\Users\003XSD744\AppData\Local\Temp\ipykernel_68264\3601875651.py:21: FutureWarning: The 

In [7]:
df.fillna(False, inplace=True)
df.head()

,Name,MAILING_DATE_MONTH_X,MAILING_DATE_DAY_X,MAILING_DATE_YEAR_X,Customer_Name_X,Customer_Address_1_X,Customer_Address_2_X,Customer_Address_City_X,CUSTOMER_ADDRESS_STATE_X,CUSTOMER_ADDRESS_ZIP_X,Premise_Street_Address_X,MEMO_1_X,MEMO_2_X,MEMO_3_X,MEMO_4_X,PREFERRED_SIGNER_NAME_X,SIGNER_TITLE_X,SPANISH_SIGNATURE_TITLE_X
0,CCSE-121-UG Lateral Hardening_Mid Construction...,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False
1,CCSE-122-UG Lateral Hardening_Project Complete...,True,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True
2,CCSE-123-UG Lateral Hardening_Project Complete...,True,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,False


In [9]:
df.to_csv('Template_Infos.csv', index=False)